# Preparação do ambiente

## Bibliotecas

In [35]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from itertools import pairwise
from pickle import load

## Constantes e funções auxiliares

In [36]:
def load_sch(sch_database_file):
    
    # load SCH database
    usecols = [0,1,11,12,13,14,15]
    dtype = {'Número de Homologação': 'str'}
    parse_dates = [0]
    date_format = '%d/%m/%Y'

    df_sch = pd.read_csv(
        sch_database_file,
        sep=';',
        usecols=usecols,
        dtype=dtype,
        parse_dates=parse_dates,
        date_format=date_format
        )

    df_sch = df_sch.sort_values(by='Data da Homologação',ascending=False)

    df_modelo = df_sch[['Número de Homologação','Modelo']].dropna()
    df_modelo = df_modelo.groupby('Número de Homologação',as_index=False)['Modelo'].apply(lambda x: ' | '.join(x))

    df_nome_comercial = df_sch[['Número de Homologação','Nome Comercial']].dropna()
    df_nome_comercial = df_nome_comercial.groupby('Número de Homologação',as_index=False)['Nome Comercial'].apply(lambda x: ' | '.join(x))

    columns_to_keep = ['Data da Homologação', 'Número de Homologação', 'Nome do Fabricante', 'Categoria do Produto', 'Tipo do Produto']
    df_sch = df_sch[columns_to_keep]
    df_sch = df_sch.drop_duplicates(subset='Número de Homologação')

    df_sch = df_sch.merge(df_modelo,how='left')
    df_sch = df_sch.merge(df_nome_comercial,how='left')
    df_sch = df_sch.fillna('')
    
    df_sch = df_sch.reset_index(drop=True)
    
    return df_sch 

In [37]:
def flatten_matrix(matrix):
    return [item for row in matrix for item in row]

In [38]:
def extract_amazon_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    imagens.extend(row['Imagens'].split())

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

def extract_amazon_desc(row):
    desc = '\n'.join(row)
    desc = desc.replace(NULL_STRING,' ')
    desc = re.sub('\s+', ' ', desc)
    return desc

In [39]:
def extract_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    
    if not pd.isna(row['Imagens']):
        imagens.extend(row['Imagens'].split(','))

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

In [40]:
def format_issue(row,remove_punct=False):
    nome,desc,img0,img1,img2 = row
    
    # remove null string
    desc = desc.replace(NULL_STRING,'')
    # remove punctiation from nome and desc
    if remove_punct:
        nome = nome.translate(str.maketrans('', '', string.punctuation))
        desc = desc.translate(str.maketrans('', '', string.punctuation))

    # remove ponto-e-vírgula e aspas duplas para evitar a 
    # identificação incorreta de campos ao importar para o redmine
    desc = desc.replace(';',',')
    desc = desc.replace('"',"''")
    # remove multiple linebrakes and spaces
    desc = re.sub('\n+','\n\n',desc)
    desc = re.sub('\s+',' ',desc)
    desc = desc.strip()
    
    img_css = '{max-width:250px; height:auto}'
    if len(img2) > 0:
        img_list = f'!{img_css}{img0}! !{img_css}{img1}! !{img_css}{img2}!'
    elif len(img1) > 0:
        img_list = f'!{img_css}{img0}! !{img_css}{img1}!'
    elif len(img0) > 0:
        img_list = f'!{img_css}{img0}!'
    else:
        img_list = ''
        
    if len(desc) >= 1:
        issue = f'h1. {nome}\n\n{desc}\n\n{img_list}'
    else:
        issue = f'h1. {nome}\n\n{img_list}'
    
    return issue.strip()

In [41]:
def plot_donut(labels,values,title,startangle=30):
    fig, ax = plt.subplots(figsize=(5, 3), subplot_kw=dict(aspect="equal"))
    
    wedges, texts = ax.pie(values, wedgeprops=dict(width=0.5), startangle=startangle)
    
    bbox_props = dict(boxstyle="round,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(arrowprops=dict(arrowstyle="-"),
              bbox=bbox_props, zorder=0, va="center")
    
    kw = dict(arrowprops=dict(arrowstyle="-"),bbox=bbox_props,
              zorder=0, va="center")
    
    for i, p in enumerate(wedges):
        ang = (p.theta2 - p.theta1)/2. + p.theta1
        y = np.sin(np.deg2rad(ang))
        x = np.cos(np.deg2rad(ang))
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = "angle,angleA=0,angleB={}".format(ang)
        kw["arrowprops"].update({"connectionstyle": connectionstyle})
        ax.annotate(labels[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                    horizontalalignment=horizontalalignment, **kw)
    
    ax.set_title(title)
    plt.show()

# Carga e prepação dos dados

## Base SCH

In [42]:
sch_database_file = r'Z:\anatel\inova-fiscaliza\certificacao-homologacao\schwebsearch\datasets\sch_database\produtos_certificados.zip'
df_sch = load_sch(sch_database_file)
df_sch.head()

,Data da Homologação,Número de Homologação,Nome do Fabricante,Categoria do Produto,Tipo do Produto,Modelo,Nome Comercial
0,2024-05-20,030572416551,Decathlon,2,Transceptor de Radiação Restrita,8605113,W900
1,2024-05-20,004732404809,Elsys Equipamentos Eletrônicos Ltda,2,Sistemas de Identificação por Radiofrequências,ESF-DE5100I | ESF-DE5100I,
2,2024-05-20,061542414894,"Shenzhen Baseus Technology Co., Ltd.",1,Acessório p/ Telefone Móvel Celular do tipo Ba...,PPAP2-10A,
3,2024-05-20,050542403757,Lear Corporation,2,Sistemas Operando nas Faixas de RF Ultra Larga,KOBJXF23A | KOBJXF23A,
4,2024-05-20,062092408867,"Fortinet, Inc.",3,Equipamento de Rede de Dados,FG-120G | FG-121G,


## Anúncios coletados

In [55]:
files = ['../datasets/inspecao_ecommerce/carrefour.parquet.gzip',   
         '../datasets/inspecao_ecommerce/ml.parquet.gzip', 
         '../datasets/inspecao_ecommerce/magalu.parquet.gzip']

files = ['../datasets/inspecao_ecommerce/carrefour.parquet.gzip']

df = pd.concat([pd.read_parquet(file) for file in files])
df = df.reset_index(drop=True)
# # df['Certificado_de_Homologação'] = df['Certificado_de_Homologação'].apply(lambda x: '-1' if pd.isna(x) else x.replace('.0','').zfill(12))
df['Descrição'] = df['Descrição'].apply(lambda x: '' if pd.isna(x) else x)

# columns_to_keep = ['Link', 'Nome', 'Descrição', 'Imagem', 'Imagens', 'Certificado_de_Homologação]
columns_to_keep = ['Link', 'Nome', 'Descrição', 'Imagem', 'Imagens']
# df = df[columns_to_keep]

df.to_excel('d:\\c.xlsx')

In [59]:
df.Palavra_Chave.value_counts()

Palavra_Chave
carregador celular    537
smartphone            488
power bank            455
bluetooth             392
wifi                  243
tv box                 18
Name: count, dtype: Int64

In [44]:
df[['Imagem', 'Imagens']].apply(lambda row: extract_magalu_images(row),axis=1)

,0,1,2
0,https://carrefourbr.vtexassets.com/arquivos/id...,,
1,https://carrefourbr.vtexassets.com/arquivos/id...,,
2,https://carrefourbr.vtexassets.com/arquivos/id...,,
3,https://carrefourbr.vtexassets.com/arquivos/id...,,
4,https://carrefourbr.vtexassets.com/arquivos/id...,,
...,...,...,...
7142,https://a-static.mlcdn.com.br/280x210/smart-tv...,https://a-static.mlcdn.com.br/90x90/smart-tv-l...,https://a-static.mlcdn.com.br/90x90/smart-tv-l...
7143,https://a-static.mlcdn.com.br/280x210/smart-tv...,https://a-static.mlcdn.com.br/90x90/smart-tv-l...,https://a-static.mlcdn.com.br/90x90/smart-tv-l...
7144,https://a-static.mlcdn.com.br/280x210/carregad...,https://a-static.mlcdn.com.br/90x90/carregador...,
7145,https://a-static.mlcdn.com.br/280x210/carregad...,https://a-static.mlcdn.com.br/90x90/carregador...,https://a-static.mlcdn.com.br/90x90/carregador...


## Modelo pré-treinado

In [5]:
file_model = '../models/clf_market_places_sgd.pkl'
with open(file_model, 'rb') as f:
    clf = load(f)
clf

Pipeline(steps=[('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 stop_words=['a', 'à', 'ao', 'aos', 'aquela',
                                             'aquelas', 'aquele', 'aqueles',
                                             'aquilo', 'as', 'às', 'até', 'com',
                                             'como', 'da', 'das', 'de', 'dela',
                                             'delas', 'dele', 'deles', 'depois',
                                             'do', 'dos', 'e', 'é', 'ela',
                                             'elas', 'ele', 'eles', ...])),
                ('transformer', TfidfTransformer()),
                ('clf', SGDClassifier(alpha=1e-05, loss='log_loss'))])

# Análise

In [6]:
# df.iloc[5013]
# df_sch[df_sch['Número de Homologação']=='076211902585']

In [60]:
docs = df['Nome'].values
passivel = clf.predict(docs)
proba = clf.predict_proba(docs)[:,1]

map_passivel = {0: 'Não', 1: 'Sim'}

df['Passível'] = passivel
df['Passível'] = df['Passível'].map(map_passivel)
df['Probabilidade'] = proba

df.head()

,Nome,Preço_Original,Preço,Imagem,Link,Data_Atualização,Palavra_Chave,Categoria,Marca,Vendedor,Desconto,Código_Produto,Descrição,Tabela_Dados,Imagens,Passível,Probabilidade
0,Controle Sem Fio De Xbox Carbon Black Wireless...,<NA>,"R$ 470,69",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/controle-sem-fio-...,2024-06-19T14:30:19,bluetooth,Consoles e Games|Acessórios gamer|Joystick e C...,<NA>,,<NA>,9296849,,{'Consoles': 'Xbox'},['https://carrefourbr.vtexassets.com/arquivos/...,Sim,0.997953
1,Headset Gamer Logitech G435 Sem Fio Bluetooth ...,<NA>,"R$ 449,90",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/headset-gamer-log...,2024-06-19T14:30:24,bluetooth,Informática|Acessórios para Informática|Fone d...,<NA>,,<NA>,317224482,,"{'Variante_Cor': 'Azul', 'Variante_Cor_Secunda...",['https://carrefourbr.vtexassets.com/arquivos/...,Sim,0.977969
2,Fone de Ouvido Headset Tectoy Bluetooth XPEAKE...,"R$ 92,60","R$ 49,98",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/fone-de-ouvido-he...,2024-06-19T14:30:32,bluetooth,Informática|Acessórios para Informática|Fone d...,<NA>,,<NA>,11325614,,"{'Variante_Cor': 'Preto', 'Modelo': 'XPEAKER -...",['https://carrefourbr.vtexassets.com/arquivos/...,Sim,0.997595
3,Capa Teclado Bluetooth Anti Impacto Ipad 9 10....,<NA>,"R$ 569,00",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/capa-teclado-blue...,2024-06-19T14:30:39,bluetooth,"Celulares, Smartphones e Smartwatches|Acessóri...",<NA>,,<NA>,326517297,,{'Dimensões aproximadas do produto (AxLxP)': '...,['https://carrefourbr.vtexassets.com/arquivos/...,Sim,0.627479
4,Multimídia Automotivo Pioneer Dmh-a248bt Webli...,<NA>,"R$ 1.197,69",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/multimidia-automo...,2024-06-19T14:30:46,bluetooth,Automotivo|Som Automotivo|Rádio Automotivo|Mul...,<NA>,,<NA>,318200096,,"{'Tipo de Veículo': 'Carro', 'Código da Homolo...",['https://carrefourbr.vtexassets.com/arquivos/...,Sim,0.971298


In [61]:
df_resumo = df.pivot_table(index='Palavra_Chave',columns='Passível', values='Nome', aggfunc='count').fillna(0)
df_resumo = df_resumo.astype('int')
df_resumo['Total'] = df_resumo.sum(axis=1)
df_resumo.loc['Total'] = df_resumo.sum()
df_resumo


Passível,Não,Sim,Total
Palavra_Chave,,,
bluetooth,2,390,392
carregador celular,61,476,537
power bank,1,454,455
smartphone,2,486,488
tv box,13,5,18
wifi,22,221,243
Total,101,2032,2133


In [10]:
columns_to_keep = ['Marca', 'Modelo', 'Nome', 'EAN', 'Certificado_de_Homologação','Passível', 'Probabilidade']
df[columns_to_keep]

,Marca,Modelo,Nome,EAN,Certificado_de_Homologação,Passível,Probabilidade
0,<NA>,<NA>,Adaptador Bluetooth De Tv Ugreen Qualcomm Aptx...,<NA>,-1,Sim,0.931037
1,<NA>,<NA>,Moto G53 5G (eSIM) 128 GB prata 4 GB RAM,<NA>,-1,Sim,0.987362
2,<NA>,<NA>,Carregador De Parede Rápido Tipo C Baseus Supe...,<NA>,-1,Sim,0.975473
3,<NA>,<NA>,Carregador De Parede Duplo Usb Baseus 10.5w + ...,<NA>,-1,Sim,0.837403
4,<NA>,<NA>,Roteador 4g Zte Mf253l 300mbps Chip Direto No ...,<NA>,-1,Sim,0.952825
...,...,...,...,...,...,...,...
5009,LG,Smart tv lg 32 hd 32lq620 wi-fi bluetooth hdr ...,Smart Tv Lg 32 Hd 32Lq620 Wi-Fi Bluetooth Hdr ...,<NA>,080281811799,Sim,0.995443
5010,<NA>,<NA>,"Smart TV LG 43'' 4K Ultra HD 43UR7800PSA, WiFi...",<NA>,-1,Sim,0.997450
5011,<NA>,<NA>,Carregador de Parede USB 2.1A 10W Bivolt - Bra...,<NA>,-1,Sim,0.940868
5012,Usams,Carregador de parede usams 20w pd com porta ti...,Carregador De Parede Usams 20W Pd Com Porta Ti...,<NA>,-1,Sim,0.953240


In [17]:
df.columns

Index(['Nome', 'Preço_Original', 'Preço', 'Imagem', 'Link', 'Data_Atualização',
       'Palavra_Chave', 'Categoria', 'Marca', 'Vendedor', 'Desconto',
       'Código_Produto', 'Descrição', 'Tabela_Dados', 'Imagens', 'Modelo',
       'EAN', 'Certificado_de_Homologação', 'Características', 'Link_Pagina',
       'Acessado', 'Qtd_vendidos', 'Qtd_disponível', 'Estado', 'Avaliações',
       'Nota'],
      dtype='object')